In [1]:
import pandas as pd
import sklearn
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
%matplotlib inline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from utils import update_submission

# RandomForest

1. Bagging (sampling with replacement)
   100 samples, randomly choose 60 out of 100, load them to Tree 1.
   Put the 60 back, resample 60, load the new 60 samples to Tree 2.
   ...
   Tree N.
2. Max_feature:
   Each split on the tree is based on one feature, each tree will have different features, maxed at Max_feature
When making predictions, all the trees will vote.

In [2]:
df_test = pd.read_csv('./data/test.csv')
df_train = pd.read_csv('./data/train.csv')
df_submission = pd.read_csv('./data/gender_submission.csv')

In [3]:
def transform(df,option):
    assert option in ["test","train"] , "Option must be test or train"
    df = df.set_index('PassengerId')
    cond = df.Fare.isnull()
    sub_value = df.Fare.mean()
    df.Fare = np.where(cond, sub_value, df.Fare)
    ticket_p = df.groupby('Ticket', as_index=False).agg({'Name': 'count'}).rename(columns={'Name':'t_count'}).sort_values('t_count', ascending=False)
    d = dict()
    for i, row in ticket_p.iterrows():
        d[row.Ticket] = row.t_count
    df_merge = df.copy()
    df_merge['t_count'] = df_merge.Ticket.apply(lambda x: d[x])
    df_merge['Fare_per_person'] = df_merge.Fare/df_merge.t_count
    cond = df_merge.Age.isnull()
    sub_value = df_merge.Age.mean()
    df_merge.Age = np.where(cond, sub_value, df_merge.Age)
    cond = (df_merge.Sex == 'female')
    sub_value = 1
    df_merge.Sex = np.where(cond, sub_value, 0)
    cond = df_merge.Embarked.isnull()
    sub_value = 'S'
    df_merge.Embarked = np.where(cond, 'S', df_merge.Embarked)
    df_merge = df_merge.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    df_merge.isnull().sum()
    ohe = OneHotEncoder()
    X_cat = ohe.fit_transform(df_merge.Embarked.values.reshape(-1,1)).toarray()
    if option=="train":
        X_short = df_merge.drop(['Survived','Embarked'], axis=1).values
    else:
        X_short = df_merge.drop(['Embarked'], axis=1).values
    X = np.hstack([X_cat, X_short])
    return X

In [6]:
X_train_val = transform(df_train, "train")
X_test = transform(df_test,"test")
y_train_val = df_train.Survived

In [5]:
rf = RandomForestClassifier()

In [8]:
cross_val_score(rf, X_train_val, y_train_val).mean()

0.8058251208335949

In [ ]:
param_grid = {'max_depth': np.arange(4,10),
              'min_samples_split': np.arange(10,20,2),
              'min_samples_leaf': np.arange(1,10,2),
              'max_features': ?????}

In [ ]:
# HW: to fine tune our RandomForestClassifier and get a accuracy > 82%.
# update the submission file with the function in utils.py, submit it to kaggle, compare the result with
# what we get locally (CV)